In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### PhoneBook_Search

In [24]:
%%writefile search_phonebook.cu
#include <bits/stdc++.h>
#include <cuda.h>

using namespace std;

// Structure representing a Contact
struct Contact {
  char id[50];
  char name[50];
  char number[50];
};

// Device function to check if two strings match
__device__ bool check(char* str1, char* str2, int len) {
  // Loop through the first string
  for(int i = 0; str1[i] != '\0'; i++) {
    int j = 0;
    // Check if substrings match character by character
    while(str1[i+j] != '\0' && str2[j] != '\0' && str1[i+j] == str2[j]) {
      j++;
    }

    if(j == len-1) {
      return true;
    }
  }
  return false;
}

// Kernel function to search the phonebook for a contact with the given name
__global__ void searchPhonebook(Contact* phonebook, int num_contacts, char* search_name, int name_length) {
  // Calculate the global index of the thread
  int idx = blockIdx.x * blockDim.x + threadIdx.x;

  // If the thread index is within the bounds of the phonebook
  if(idx < num_contacts) {
    if(check(phonebook[idx].name, search_name, name_length)) {
      printf("%s %s %s\n", phonebook[idx].id, phonebook[idx].name, phonebook[idx].number);
    }
  }
}

int main(int argc, char* argv[]) {
  // Ensure correct number of command-line arguments
  if(argc != 3) {
    cerr << "Usage: " << argv[0] << " <search_name> <num_threads>" << endl;
    return 1;
  }

  string search_name = argv[1];  // Search name provided as argument
  int num_threads = atoi(argv[2]);  // Number of threads per block

  // Path to the phonebook file
  string file_name = "/content/drive/MyDrive/Parallel_Processing/phonebook.txt";

  vector<Contact> phonebook;

  // Open the phonebook file
  ifstream file(file_name);
  if(!file.is_open()) {
    cerr << "Error opening file: " << file_name << endl;
    return 1;
  }
  else {
    Contact contact;
    string line;

    // Read each line from the phonebook file
    while(getline(file, line)) {
      int pos = line.find(",");
      // Extract the name (without quotes)
      strcpy(contact.name, line.substr(1, pos - 2).c_str());
      // Extract the phone number (without quotes)
      strcpy(contact.number, line.substr(pos + 2, line.size() - pos - 4).c_str());

      phonebook.push_back(contact);
    }
    file.close();
  }

  int num_contacts = phonebook.size();

  // Allocate memory on the GPU for the phonebook
  Contact* device_phonebook;
  cudaMalloc((void**)&device_phonebook, sizeof(Contact)*num_contacts);

  // Copy the phonebook data from host to device
  cudaMemcpy(device_phonebook, phonebook.data(), sizeof(Contact)*num_contacts, cudaMemcpyHostToDevice);

  // Allocate memory on the GPU for the search name
  int name_length = search_name.length() + 1;
  char* device_search_name;
  cudaMalloc((void**)&device_search_name, name_length);

  // Copy the search name from host to device
  cudaMemcpy(device_search_name, search_name.c_str(), name_length, cudaMemcpyHostToDevice);

  // Launch the kernel with multiple blocks, processing `num_threads` contacts at a time
  for(int i = 0; i < num_contacts; i += num_threads) {
    int thread_count = min(num_contacts-i, num_threads);
    searchPhonebook<<<1, thread_count>>>(device_phonebook + i, thread_count, device_search_name, name_length);
    cudaDeviceSynchronize();
  }

  // Free the allocated memory on the device
  cudaFree(device_phonebook);
  cudaFree(device_search_name);

  return 0;
}


Overwriting search_phonebook.cu


In [25]:
!nvcc -arch=sm_75 search_phonebook.cu -o search_phonebook

In [31]:
!time ./search_phonebook JAHAN 200 > output.txt


real	0m0.140s
user	0m0.015s
sys	0m0.118s
